In [1]:
import numpy as np

In [2]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [12]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.077625769483447, 10.0)

In [13]:
default_time

array([  5.64288864,   8.57712808,   2.06037537, ...,  23.07033775,
        12.18076758,   1.25239857])

In [52]:
# Put your code here to price the corporate bond

def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000):
    U = np.random.uniform(size=n_sample)
    default_time = -(1/def_rate)*np.log(U)
    vals1 = np.zeros(n_sample)
    for i in range(0,9999):
        if default_time[i] > 1:
            vals1[i] = np.exp(-mat*rf_rate)
        else :
            vals1[i] = recovery*np.exp(-default_time[i]*rf_rate) 
    price = vals1.mean()
    return price



# Call your function
BondPrice1 = np.zeros(100)
for j in range(100):
    BondPrice1[j] = corp_bond()
     
(BondPrice1.mean(),BondPrice1.std())    
    

# Find the mean and std by calling the function 100 times.

(0.94104114320239285, 0.0012844551919007957)

In [47]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.9999999999999982, 10.0)

In [51]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=10000, antithetic=True, mean_match=True):
    U = np.random.uniform(size=n_sample)
    if(antithetic):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        vals2 = np.zeros(n_sample)
        for i in range(0,9999):
            if default_time[i] > 1:
                vals2[i] = np.exp(-mat*rf_rate)
            else :
                vals2[i] = recovery*np.exp(-default_time[i]*rf_rate) 
        price = vals2.mean()
    if(mean_match):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        default_time += 1/def_rate-default_time.mean()
        vals3 = np.zeros(n_sample)
        for i in range(0,9999):
            if default_time[i] > 1:
                vals3[i] = np.exp(-mat*rf_rate)
            else :
                vals3[i] = recovery*np.exp(-default_time[i]*rf_rate) 
        price = vals3.mean() 
    return price

BondPrice2 = np.zeros(100)
for j in range(100):
    BondPrice2[j] =  corp_bond_cv(mean_match=False)
     
print(BondPrice2.mean(),BondPrice2.std())    

BondPrice3 = np.zeros(100)
for j in range(100):
    BondPrice3[j] =  corp_bond_cv(antithetic=False)
     
print(BondPrice3.mean(),BondPrice3.std()) 

# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

0.941241285326 0.00113823150698
0.941566755257 0.00298940002001


In [50]:
analyticvalue = 0
K = np.arange(0,1,0.0001)
for k in range(0,10000):
    m = K[j]
    analyticvalue += 0.3*np.dot(m,np.exp(-0.04*m))*0.03*np.exp(-0.04*m)

analyticvalue += np.dot(1-np.exp(-0.03),np.exp(-0.04))

analyticvalue    

0.91938856256018919

In [53]:
print(BondPrice1.mean()-analyticvalue)
print(BondPrice2.mean()-analyticvalue)
print(BondPrice3.mean()-analyticvalue)

0.0216525806422
0.0218527227654
0.0221781926967
